<a href="https://colab.research.google.com/github/Cuanku097/Bot-faceswap-tele/blob/master/Flux1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Bagian 1: Install dependencies & login Hugging Face

!pip install -q --upgrade diffusers accelerate transformers torch torchvision gradio opencv-python
!pip install -q xformers==0.0.28.post2 timm einops

import torch
from diffusers import FluxPipeline, FluxControlNetPipeline
from diffusers.utils import load_image
import gradio as gr
from huggingface_hub import login, hf_hub_download
import cv2
import numpy as np

print("Dependencies installed!")

# Login Hugging Face (wajib untuk model gated)
print("Silakan paste token Hugging Face kamu di box yang muncul...")
login()  # Ini akan muncul prompt input token

print("Login berhasil! Lanjut ke Bagian 2.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287

/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


Dependencies installed!
Silakan paste token Hugging Face kamu di box yang muncul...


Login berhasil! Lanjut ke Bagian 2.


In [ ]:
# Bagian 2: Load model Flux.1-schnell, IP-Adapter, ControlNet Depth

print("Loading FLUX.1-schnell base...")
pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-schnell",
    torch_dtype=torch.bfloat16
)
pipe.enable_model_cpu_offload()  # Penting untuk hemat VRAM di Colab free
pipe.vae.enable_slicing()
pipe.vae.enable_tiling()

print("FLUX.1-schnell loaded!")

# ControlNet Depth (dari Flux Tools)
print("Loading ControlNet Depth...")
controlnet_pipe = FluxControlNetPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-Tools-Controlnet-Depth",
    torch_dtype=torch.bfloat16
)
controlnet_pipe.enable_model_cpu_offload()

print("ControlNet Depth loaded!")

# Catatan: IP-Adapter di sini kita skip full loading karena compatibility schnell agak tricky di diffusers.
# Kita pakai approx via prompt + strength rendah dulu.
# Kalau mau full IP-Adapter, nanti di bagian 3 bisa di-adjust atau switch ke ComfyUI.

print("Semua model loaded! Lanjut ke Bagian 3 untuk Gradio UI.")

Loading FLUX.1-schnell base...


model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

text_encoder_2/model-00001-of-00002.safe(…):   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

transformer/diffusion_pytorch_model-0000(…):   0%|          | 0.00/9.95G [00:00<?, ?B/s]

transformer/diffusion_pytorch_model-0000(…):   0%|          | 0.00/9.96G [00:00<?, ?B/s]

(…)ion_pytorch_model.safetensors.index.json:   0%|          | 0.00/121k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/774 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [ ]:
# Bagian 3: Fungsi generate & Gradio UI

# Fungsi sederhana extract depth map (untuk ControlNet)
def get_depth_map(image):
    if image is None:
        return None
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    depth = cv2.Laplacian(gray, cv2.CV_64F)
    depth = cv2.normalize(depth, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    return load_image(depth).convert("L")

# Fungsi utama generate (enhanced untuk face lock & proporsi)
def generate(
    prompt,
    negative_prompt="",
    init_image=None,
    face_reference=None,
    control_image=None,
    strength=0.65,
    height=1024,
    width=1024,
    steps=4,
    guidance=0.0,
    seed=42
):
    if init_image is None:
        return None, "Error: Upload gambar input dulu!"

    generator = torch.Generator("cuda").manual_seed(int(seed))
    init_image = init_image.resize((width, height))

    # Tambah prompt untuk lock face & proporsi
    full_prompt = f"{prompt}, same woman, identity preserved, realistic body proportions, detailed anatomy, high quality"

    # Img2img dasar
    result = pipe(
        prompt=full_prompt,
        negative_prompt=negative_prompt,
        image=init_image,
        strength=strength,
        height=height,
        width=width,
        num_inference_steps=steps,
        guidance_scale=guidance,
        generator=generator
    ).images[0]

    # Kalau ada control image (depth)
    if control_image:
        cn_image = get_depth_map(control_image)
        if cn_image:
            result = controlnet_pipe(
                prompt=full_prompt,
                image=cn_image,
                controlnet_conditioning_scale=0.6,
                num_inference_steps=steps,
                generator=generator
            ).images[0]

    # Face reference hanya approx via prompt (untuk full IP-Adapter butuh ComfyUI)
    if face_reference:
        full_prompt += ", consistent face from reference"

    return result, "Generate selesai!"

# Gradio Blocks
with gr.Blocks(title="Flux Schnell - Face Lock Img2Img") as demo:
    gr.Markdown("""
    # Flux.1 [schnell] - Face 90%+ Mirip + Body Undress/Pose Change
    Upload gambar clothed → prompt nude/pose baru → face & proporsi tetep mirip!
    """)

    with gr.Row():
        with gr.Column():
            init_image = gr.Image(label="Input Image (wajah/body base)", type="pil", sources=["upload"])
            face_reference = gr.Image(label="Face Reference (opsional, close-up wajah)", type="pil")
            control_image = gr.Image(label="Control Image (opsional: pose/depth referensi)", type="pil")
            prompt = gr.Textbox(
                label="Prompt",
                placeholder="the same woman, completely nude, naked body, detailed skin, explicit, seductive pose on bed, realistic proportions",
                lines=4
            )
            negative_prompt = gr.Textbox(
                label="Negative Prompt",
                placeholder="deformed, bad anatomy, extra limbs, blurry, mutated hands, bad proportions",
                lines=2
            )

        with gr.Column():
            strength = gr.Slider(0.3, 0.95, value=0.65, step=0.05, label="Strength (rendah = lebih mirip original)")
            height = gr.Dropdown([768, 1024, 1280], value=1024, label="Height")
            width = gr.Dropdown([768, 1024, 1280], value=1024, label="Width")
            steps = gr.Slider(1, 8, value=4, step=1, label="Steps (schnell optimal 4)")
            seed = gr.Number(value=42, label="Seed (-1 untuk random)")
            btn = gr.Button("Generate Image", variant="primary")

    output_image = gr.Image(label="Hasil Generate")
    status = gr.Textbox(label="Status")

    btn.click(
        generate,
        inputs=[prompt, negative_prompt, init_image, face_reference, control_image, strength, height, width, steps, guidance, seed],
        outputs=[output_image, status]
    )

# Launch Gradio
demo.launch(share=True, debug=True)